In [1]:
# import
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random

In [2]:
'''
Represents a Stochastic Maze problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticMazeEnv(gym.Env):
    '''
    StochasticMazeEnv represents the Gym Environment for the Stochastic Maze environment
    States : [0,1,2,3,4,5,6,7,8,9,10,11]
    Actions : ["Left":0, "Up":1, "Right":2, "Down":3]
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=0,no_states=12,no_actions=4):
        '''
        Constructor for the StochasticMazeEnv class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 12
            no_actions : The no. of possible actions which is 4
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.actions_dict = {"L":0, "U":1, "R":2, "D":3}
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        index = np.random.choice(3,1,p=[0.8,0.1,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

In [3]:
env = StochasticMazeEnv()
env.reset()
num_states = env.nS
num_actions = env.nA

## Test Cases for checking the environment 

## Example: Random Policy

In [4]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    rand_action = np.random.choice(4,1)[0]  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(rand_action)
    
    total_reward += reward

    print("  ", rand_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   1 	   2 	 -0.01 	   False
   3 	   6 	 -0.01 	   False
   3 	   10 	 -0.01 	   False
   1 	   6 	 -0.01 	   False
   2 	   7 	 -1 	   True
Total Number of steps to Reach Terminal: 6
Final Reward: -1.05


### The random policy takes large number of steps to reach some terminal state which should be much higher than the number of the steps taken by a all 'Right' policy

## Example: Right Policy

In [6]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    right_action = 2  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(right_action)
    
    total_reward += reward

    print("  ", right_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   1 	 -0.01 	   False
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   6 	 -0.01 	   False
   2 	   10 	 -0.01 	   False
   2 	   11 	 -0.01 	   False
   2 	   11 	 -0.01 	   False
   2 	   11 	 -0.01 	   False
   2 	   11 	 -0.01 	   False
   2 	   11 	 -0.01 	   False
   2 	   11 	 -0.01 	   False
   2 	   11 	 -0.01 	   False
   2 	   11 	 -0.01 	   False
   2 	   7 	 -1 	   True
Total Number of steps to Reach Terminal: 15
Final Reward: -1.14


### The right policy most of the time reaches the goal state in just 3 steps which is expected

***

## Q1. Find an optimal policy to navigate the given environment using Policy Iteration 

In [6]:
# write your code here
# hint: code policy evaluation function
# hint: code policy improvement function
# hint: implement policy iteration

# import
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random

'''
Represents a Stochastic Maze problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticMazeEnv(gym.Env):
    '''
    StochasticMazeEnv represents the Gym Environment for the Stochastic Maze environment
    States : [0,1,2,3,4,5,6,7,8,9,10,11]
    Actions : ["Left":0, "Up":1, "Right":2, "Down":3]
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=0,no_states=12,no_actions=4):
        '''
        Constructor for the StochasticMazeEnv class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 12
            no_actions : The no. of possible actions which is 4
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.actions_dict = {"L":0, "U":1, "R":2, "D":3}
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        index = np.random.choice(3,1,p=[0.8,0.1,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]
    
    def get_transition_probabilities(self, state, action):
        if state in self.prob_dynamics and action in self.prob_dynamics[state]:
            return self.prob_dynamics[state][action]
        else: 
            #handling the case of state-action pair(5th state) where transition probabilities are not defined 
            #returns an empty list
            return []
        

class PolicyIteration:
    def __init__(self, env , gamma = 0.9, epsilon = 1e-6):
        self.env = env #stochastic maze
        self.gamma = gamma #discount factor
        self.epsilon = epsilon #convergence threshold
        self.policy = {s: random.randint(0, 3) for s in range(12)} #initializing with random actions between 0,1,2,3 
        self.V = {s: 0 for s in range(12)} # intializing value function to 0  

    def policy_evaluation(self):  
        #evaluates value function for a given policy until it converges
        while True:
            d = 0 
            for s in range(12):
                #updating value function
                v = self.V[s] 
                new_v = 0 #updated value fos state s
                for a in range(4): # 4 actions
                    transition = self.env.get_transition_probabilities(s, a) #retrieving transition probabilities
                    for prob,next_s, reward, _ in transition:
                        new_v += prob * (reward + self.gamma * self.V[next_s]) #bellman equation for value function
                self.V[s] = new_v #updating value function
                d = max(d, np.abs(v - self.V[s])) #max change in value function across all states
            if d < self.epsilon:
                #converges or falls below specified thresold
                break 

     
    def policy_improvement(self):
        policy_stable = True
        for s in range(12):
            old_action = self.policy[s]
            action_values = []
            for a in range(4):
                transitions = self.env.get_transition_probabilities(s, a)
                action_value = sum(prob *(reward + self.gamma * self.V[next_s]) for prob, next_s, reward, _ in transitions) #Bellman equation for action-value function
                action_values.append(action_value)
            #greedily selecting the action with highest action value for each state
            if max(action_values) <= 0: #all action values are negative
                self.policy[s] = random.randint(0, 3)
            else:
                self.policy[s] = action_values.index(max(action_values))  #maximum value as action value
            if old_action!= self.policy[s]:
                policy_stable = False #policy has changed
        return policy_stable  #determinies if policy as improved or not
    
    def policy_iteration(self):
        #finding optimal policy 
        #iterating the above two steps
        while True:
            self.policy_evaluation()
            if self.policy_improvement():
                break
            #Stop iteration only when poicy remains stable i.e unchanged 
        return self.policy
    
    
env = StochasticMazeEnv()
policy_iter = PolicyIteration(env)
optimal_policy = policy_iter.policy_iteration()
print("Optimal Policy: ")
print(optimal_policy)



Optimal Policy: 
{0: 0, 1: 3, 2: 1, 3: 2, 4: 1, 5: 2, 6: 0, 7: 0, 8: 1, 9: 1, 10: 3, 11: 3}


## Q2. Find an optimal policy to navigate the given environment using Value Iteration

In [7]:
# write your code here
class ValueIteration:
    def __init__(self, env, gamma=0.9, epsilon=1e-6):
        self.env = env #stochastic maze
        self.gamma = gamma  # discount factor
        self.epsilon = epsilon #convergence threshold
        self.V = {s: 0 for s in range(12)}  # value function

    def value_iteration(self):
        while True:
            d = 0
            for s in range(12):
                v = self.V[s]
                action_values = []
                for a in range(4):
                    transitions = self.env.get_transition_probabilities(s, a)
                    action_value = sum(prob * (reward + self.gamma * self.V[next_s]) for prob, next_s, reward, _ in transitions) #bellman equation
                    action_values.append(action_value)
                self.V[s] = max(action_values) if action_values else 0
                d = max(d, abs(v - self.V[s]))
            if d < self.epsilon: #converge  
                break
        # Extract policy from the value function
        optimal_policy = {}
        for s in range(12):
            action_values = []
            for a in range(4):
                transitions = self.env.get_transition_probabilities(s, a)
                action_value = sum(prob * (reward + self.gamma * self.V[next_s]) for prob, next_s, reward, _ in transitions)
                action_values.append(action_value)
            optimal_policy[s] = action_values.index(max(action_values)) #maximum action value
        return optimal_policy
    
env = StochasticMazeEnv()
value_iter = ValueIteration(env) 
optimal_policy = value_iter.value_iteration()
print("Optimal Policy:") 
print(optimal_policy)

Optimal Policy:
{0: 2, 1: 2, 2: 2, 3: 0, 4: 1, 5: 0, 6: 1, 7: 0, 8: 1, 9: 2, 10: 1, 11: 0}


## Q3. Compare PI and VI in terms of convergence. Is the policy obtained by both same?

### Answer:  The optimal policies obtained by Policy Iteration and Value Iteration are slightly different for the above stochastic environment, as convergence rates, exploration rates of state-action space, algorithms, and update rules are different. Value iteration updates the value function iteratively until it converges whereas policy iteration iterates between policy evaluation and policy improvement steps. PI converges faster than VI due to its iteration between policy evaluation and improvement, while VI requires some iterations to converge. As number of iterations are not being specified in policy iteration, it takes more time to run, while value iteration does a single iteration of policy evaluation at each step. 